In [4]:
import sys
sys.path.append('../')
import torch
import os, glob
import json
from re import finditer
import pickle
import csv 

In [5]:
folder = "../output"
out_dir = folder + "/Ontologies_list"
ontologies_dir = out_dir + "/json/*"
output_dir = out_dir + "/pkl_list/"

In [ ]:
vocab = torch.load(f"{folder}/vocab.pt")
vocab_stoi = vocab.get_stoi()
vocab_list = set(vocab_stoi.keys())
# print(vocab_list)

In [42]:
def camel_case_split(identifier):
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0).lower() for m in matches]      

In [43]:

for filename in glob.iglob(ontologies_dir, recursive=True):
    if os.path.isfile(filename):
        concepts= set()
        with open(filename,encoding = "ISO-8859-1") as f:
            data = json.load(f)
            for attribute in data["classAttribute"]:
                if "label" in attribute and "IRI-based" in attribute["label"]:
                    concepts.add(" ".join(camel_case_split(attribute["label"]["IRI-based"])))
            
            name = os.path.basename(filename)
            out = output_dir + name.split(".json")[0] + ".pkl"
            with open(out,"wb") as f: 
                pickle.dump(concepts,f)
                

In [79]:
def compare_vocab_with_ontologies(concepts, filename):
    excluded_from_vocab = set()
    excluded_from_vocab1 = set()
    
    for item in concepts:
        elem = item.split(" ")
        elem_len = len(elem)
        elem_counter = 0
        for indiv in elem:
            if indiv in vocab_list:
                elem_counter +=1
            else:
                excluded_from_vocab1.add(indiv)
                
        if elem_counter != elem_len:
            excluded_from_vocab.add(item)
    
    write_output_to_csv(concepts, True, excluded_from_vocab, filename)
    
    write_output_to_csv(concepts, False, excluded_from_vocab1, filename)


In [ ]:
def write_output_to_csv(concepts, all_words, excluded_from_vocab, filename):
    out_filename = out_dir + "/report.csv"
    
    with open(out_filename, "a") as f:
        writer = csv.writer(f)
        text = "Compared with keywords as it is" if all_words else "Compared with each word of a particular keyword"

        row = [filename, text , len(vocab_list), len(concepts), len(excluded_from_vocab), str(excluded_from_vocab)]
        writer.writerow(row)

In [83]:

for filename in glob.iglob(output_dir + "*", recursive=True):
    if os.path.isfile(filename):
        concepts= set()
        name = os.path.basename(filename)
        with open(filename,"rb") as f:
            concepts = pickle.load(f)
            compare_vocab_with_ontologies(concepts, name)
            
print("Task Completed")

Task Completed
